In [1]:
import requests
# from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import pickle
import time

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from collections import defaultdict

import pipeline as p

%load_ext autoreload
%autoreload 2

In [2]:
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

chromedriver = "/Users/kendra/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url_base = 'https://www.pro-football-reference.com'

team_dict = p.open_pkl('TeamDicts/team_dictionary.pkl')

Create a dictionary to track which teams & seasons we've scraped data for

In [8]:
# season_dict:
    # key = [year] 
    # vals = list of teams whose team-season page has been sucessfully scraped

# season_dict = p.open_pkl('Data/season_dict.pkl')    # use this if need to re-start scraping
season_dict = defaultdict(list)  # [year] = list of teams whose team-season page has been sucessfully scraped

### Iterate over team-season pages, and boxscores within a given season:

In [ ]:
driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)
# driver = webdriver.Chrome(chromedriver)

for year in range(2018, 2019):
    print(year)
    bxsc_html = defaultdict(list)    # [href] = html -- also tracks whether a bxsc href has been visited
    bxsc_refs = {}                   # [team-year] = list of hrefs for boxscores
    season_html = {}                 # [team-year] = html of team season page
    
    for tm_list in team_dict.values():
        team, url = tm_list[1], tm_list[2]
        tm_yr = team +'-' + str(year)
        if team not in season_dict[year]:

            # get html of season data table
            html = p.get_season_data(url, year, driver)

            # if successfully returned data:
            if html:
                # add html to season_html dictionary
                season_html[tm_yr] = html

                # generate list of individaul game boxscore URLs, add them to bxsc_refs dictionary
                href_list = p.get_href_list(html)
                bxsc_refs[tm_yr] = href_list

                # get & add bxsc html's to bxsc_html dict
                for a in href_list:
                    href = a.attrs['href']
                    if href not in bxsc_html.keys():
                        bxsc_html[href] = p.get_bxsc_data(href, driver)

                # add team & year to season dictionary
                season_dict[year].append(team)

#     p.pkl_this('bxsc_refs_' + str(year) + '.pkl',bxsc_refs)
    p.save_dict_txt('Data/bxsc_refs_' + str(year) + '.txt', bxsc_refs)
    p.pkl_this('Data/bxsc_html_' + str(year) + '.pkl', bxsc_html)   # changed this to include year after I ran
    p.pkl_this('Data/season_html_' + str(year) + '.pkl', season_html)
    p.pkl_this('Data/season_dict.pkl', season_dict)
    
p.close_dr()

2018
Loading page for nwe 2018
Page loaded & data found!


### Elo Data from FiveThirthyEight

File downloaded from: https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv

In [ ]:
import datetime as dt

In [ ]:
elo_df = pd.read_csv('Data/nfl_games.csv')
elo_df['date'] = pd.to_datetime(elo_df['date'], format='%Y-%m-%d')

# Can drop anything before 2007
elo_df = elo_df[elo_df['date'] >= dt.datetime(2007,8,1)].reset_index(drop=True)

p.pkl_this('Data/elo_df.pkl', elo_df)